# Train model and do test

In [110]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from Hyperparameter_optimization import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [111]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
feature_names = load_headers(DATA_TRAIN_PATH)

# convert y from -1/1 to 0,1
y = (y+1)/2

In [112]:
from preprocess_data import *
data_pro  = data_preprocess_train(tX,y,outlier_method = 'IQR',n_out = 3)

## Without bias

In [113]:
from Hyperparameter_optimization import *

In [114]:
lambda_ = [0]
gamma_ = [1e-2]
accuracy_ = []
group_size_ = []
for idx_data,data in enumerate(data_pro):
    print('┌'+'─' * 38+'┐')
    print('│               Group: %i               │'%idx_data)
    print('└'+'─' * 38+'┘')
    tx = data[0]
    y = data[1]
    initial_w_ = np.ones(tx.shape[1])
    optimal_accuracy, optimal_lambda_, optimal_gamma = Hyperparameter_optimization( y, tx,k_fold=4, 
                                                                                   lambdas=lambda_ , 
                                                                                   gammas = gamma_, 
                                                                                   initial_w = initial_w_, 
                                                                                   max_iters = 100, 
                                                                                   model = 'least_squares')
    accuracy_.append(optimal_accuracy)
    group_size_.append(len(data[2]))

accuracy_all = np.dot(accuracy_,group_size_)/np.sum(group_size_)
print('\n * Overall Accuracy = %.4f'%accuracy_all)

┌──────────────────────────────────────┐
│               Group: 0               │
└──────────────────────────────────────┘
────────────────────────────────────────
lambda : 0,  gamma: 0.01 
 
 Model least_squares 
 Loss: 0.13134087391240726 
 Accuracy: 0.6929717569252453
════════════════════════════════════════
*The optimal hyperparameters*:
 Model: least_squares 
 Optimal accuracy: 0.6929717569252453 
 Optimal lambda_: 0 
 Optimal gamma: 0.01 
 
┌──────────────────────────────────────┐
│               Group: 1               │
└──────────────────────────────────────┘
────────────────────────────────────────
lambda : 0,  gamma: 0.01 
 
 Model least_squares 
 Loss: 0.026579363200754248 
 Accuracy: 0.9403139356814703
════════════════════════════════════════
*The optimal hyperparameters*:
 Model: least_squares 
 Optimal accuracy: 0.9403139356814703 
 Optimal lambda_: 0 
 Optimal gamma: 0.01 
 
┌──────────────────────────────────────┐
│               Group: 2               │
└──────────────

## With bias

In [116]:
lambda_ = [0]
gamma_ = [1e-2]
accuracy_ = []
group_size_ = []
for idx_data,data in enumerate(data_pro):
    print('┌'+'─' * 38+'┐')
    print('│               Group: %i               │'%idx_data)
    print('└'+'─' * 38+'┘')
    tx = data[0]
    tx = np.hstack((np.ones((tx.shape[0],1)), tx))
    y = data[1]
    initial_w_ = np.ones(tx.shape[1])
    optimal_accuracy, optimal_lambda_, optimal_gamma = Hyperparameter_optimization( y, tx,k_fold=4, 
                                                                                   lambdas=lambda_ , 
                                                                                   gammas = gamma_, 
                                                                                   initial_w = initial_w_, 
                                                                                   max_iters = 100, 
                                                                                   model = 'least_squares')
    accuracy_.append(optimal_accuracy)
    group_size_.append(len(data[2]))

accuracy_all = np.dot(accuracy_,group_size_)/np.sum(group_size_)
print('\n * Overall Accuracy = %.4f'%accuracy_all)

┌──────────────────────────────────────┐
│               Group: 0               │
└──────────────────────────────────────┘
────────────────────────────────────────
lambda : 0,  gamma: 0.01 
 
 Model least_squares 
 Loss: 0.07874508171824787 
 Accuracy: 0.7775925624762834
════════════════════════════════════════
*The optimal hyperparameters*:
 Model: least_squares 
 Optimal accuracy: 0.7775925624762834 
 Optimal lambda_: 0 
 Optimal gamma: 0.01 
 
┌──────────────────────────────────────┐
│               Group: 1               │
└──────────────────────────────────────┘
────────────────────────────────────────
lambda : 0,  gamma: 0.01 
 
 Model least_squares 
 Loss: 0.02479975684777088 
 Accuracy: 0.9402373660030628
════════════════════════════════════════
*The optimal hyperparameters*:
 Model: least_squares 
 Optimal accuracy: 0.9402373660030628 
 Optimal lambda_: 0 
 Optimal gamma: 0.01 
 
┌──────────────────────────────────────┐
│               Group: 2               │
└───────────────

## With bias and second order 

In [123]:
lambda_ = [0]
gamma_ = [1e-2]
accuracy_ = []
group_size_ = []
for idx_data,data in enumerate(data_pro):
    print('┌'+'─' * 38+'┐')
    print('│               Group: %i               │'%idx_data)
    print('└'+'─' * 38+'┘')
    tx = data[0]
    tx = np.hstack((np.ones((tx.shape[0],1)), tx,tx**2))
    y = data[1]
    initial_w_ = np.ones(tx.shape[1])
    optimal_accuracy, optimal_lambda_, optimal_gamma = Hyperparameter_optimization( y, tx,k_fold=4, 
                                                                                   lambdas=lambda_ , 
                                                                                   gammas = gamma_, 
                                                                                   initial_w = initial_w_, 
                                                                                   max_iters = 100, 
                                                                                   model = 'least_squares')
    accuracy_.append(optimal_accuracy)
    group_size_.append(len(data[2]))

accuracy_all = np.dot(accuracy_,group_size_)/np.sum(group_size_)
print('\n * Overall Accuracy = %.4f'%accuracy_all)

┌──────────────────────────────────────┐
│               Group: 0               │
└──────────────────────────────────────┘
────────────────────────────────────────
lambda : 0,  gamma: 0.01 
 
 Model least_squares 
 Loss: 0.0728864008015079 
 Accuracy: 0.7952377080284058
════════════════════════════════════════
*The optimal hyperparameters*:
 Model: least_squares 
 Optimal accuracy: 0.7952377080284058 
 Optimal lambda_: 0 
 Optimal gamma: 0.01 
 
┌──────────────────────────────────────┐
│               Group: 1               │
└──────────────────────────────────────┘
────────────────────────────────────────
lambda : 0,  gamma: 0.01 
 
 Model least_squares 
 Loss: 0.02309335642638366 
 Accuracy: 0.9424961715160797
════════════════════════════════════════
*The optimal hyperparameters*:
 Model: least_squares 
 Optimal accuracy: 0.9424961715160797 
 Optimal lambda_: 0 
 Optimal gamma: 0.01 
 
┌──────────────────────────────────────┐
│               Group: 2               │
└────────────────

### Try higher orders and non-linear term

In [130]:
lambda_ = [0]
gamma_ = [1e-2]
accuracy_ = []
group_size_ = []
for idx_data,data in enumerate(data_pro):
    print('┌'+'─' * 38+'┐')
    print('│               Group: %i               │'%idx_data)
    print('└'+'─' * 38+'┘')
    tx = data[0]
    tx = np.hstack((np.ones((tx.shape[0],1)), tx,tx**2,tx**3,np.sin(tx)))
    y = data[1]
    initial_w_ = np.ones(tx.shape[1])
    optimal_accuracy, optimal_lambda_, optimal_gamma = Hyperparameter_optimization( y, tx,k_fold=4, 
                                                                                   lambdas=lambda_ , 
                                                                                   gammas = gamma_, 
                                                                                   initial_w = initial_w_, 
                                                                                   max_iters = 100, 
                                                                                   model = 'least_squares')
    accuracy_.append(optimal_accuracy)
    group_size_.append(len(data[2]))

accuracy_all = np.dot(accuracy_,group_size_)/np.sum(group_size_)
print('\n * Overall Accuracy = %.4f'%accuracy_all)

┌──────────────────────────────────────┐
│               Group: 0               │
└──────────────────────────────────────┘
────────────────────────────────────────
lambda : 0,  gamma: 0.01 
 
 Model least_squares 
 Loss: 0.07084674034929066 
 Accuracy: 0.801553098064726
════════════════════════════════════════
*The optimal hyperparameters*:
 Model: least_squares 
 Optimal accuracy: 0.801553098064726 
 Optimal lambda_: 0 
 Optimal gamma: 0.01 
 
┌──────────────────────────────────────┐
│               Group: 1               │
└──────────────────────────────────────┘
────────────────────────────────────────
lambda : 0,  gamma: 0.01 
 
 Model least_squares 
 Loss: 0.022211711691992844 
 Accuracy: 0.9464012251148546
════════════════════════════════════════
*The optimal hyperparameters*:
 Model: least_squares 
 Optimal accuracy: 0.9464012251148546 
 Optimal lambda_: 0 
 Optimal gamma: 0.01 
 
┌──────────────────────────────────────┐
│               Group: 2               │
└────────────────